In [1]:
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq

# Prepare data

In [2]:
%%time
df = pq.read_table('data/competition_data_final_pqt').select(['user_id', 'url_host', 'request_cnt', 'part_of_day', 'date']).to_pandas()

CPU times: user 1min 20s, sys: 1min 28s, total: 2min 49s
Wall time: 28.6 s


In [3]:
df.shape

(322899435, 5)

In [4]:
list(df.columns)

['user_id', 'url_host', 'request_cnt', 'part_of_day', 'date']

In [5]:
import tqdm

print("Unique values in column.")
for c in df.columns:
    if c != 'user_id':
        print(c, ":", len(df[c].unique()))

Unique values in column.
url_host : 199683
request_cnt : 15
part_of_day : 4
date : 396


In [6]:
df.part_of_day.value_counts()

day        107328399
evening     96239286
morning     85236015
night       34095735
Name: part_of_day, dtype: int64

### Impute price, create date+day_part event_time

In [7]:
df['event_time'] = pd.to_datetime(df["date"]).values.astype('datetime64[h]').view('int64')

In [8]:
df['event_time'] += df['part_of_day'].map({'morning': 0, 'day': 6, 'evening': 12, 'night': 18})

In [9]:
df.head()

,user_id,url_host,request_cnt,part_of_day,date,event_time
0,45098,ad.adriver.ru,1,morning,2022-06-15,459792
1,45098,apple.com,1,morning,2022-06-19,459888
2,45098,avatars.mds.yandex.net,1,day,2022-06-12,459726
3,45098,googleads.g.doubleclick.net,1,day,2022-05-16,459078
4,45098,googleads.g.doubleclick.net,1,day,2022-05-30,459414


# Split url_host by level

In [10]:
df['url_host_1'] = df['url_host'].apply(lambda x: x.split(".")[-1])
df['url_host_2'] = df['url_host'].apply(lambda x: x.split(".")[-2] if len(x.split("."))>1 else '_' )
df['url_host_3'] = df['url_host'].apply(lambda x: ".".join(x.split(".")[:-2]) if len(x.split("."))>2 else '_' )
del df['url_host']

# Create transactional data

In [11]:
import tqdm

print("Unique values in column.")
for c in df.columns:
    if c != 'user_id':
        print(c, ":", len(df[c].unique()))

Unique values in column.
request_cnt : 15
part_of_day : 4
date : 396
event_time : 1584
url_host_1 : 869
url_host_2 : 125474
url_host_3 : 48933


In [31]:
df.columns

Index(['user_id', 'request_cnt', 'part_of_day', 'event_time', 'url_host_1',
       'url_host_2', 'url_host_3'],
      dtype='object')

In [30]:
del df['date']

In [32]:
from ptls.preprocessing import PandasDataPreprocessor

preprocessor = PandasDataPreprocessor(
    col_id='user_id',
    col_event_time='event_time',
    event_time_transformation='none',
    cols_category=['url_host_1', 'url_host_2', 'url_host_3', 'part_of_day'],
    cols_numerical=['request_cnt'],
    return_records=False,
)

In [33]:
%%time
import pickle

trans = preprocessor.fit_transform(df)

with open('preprocessor_split.p', 'wb') as f:
    pickle.dump(preprocessor, f)

CPU times: user 10min 23s, sys: 5min, total: 15min 23s
Wall time: 15min 23s


In [34]:
%%time

import pickle
import numpy as np


cols = ['url_host_1', 'url_host_2', 'url_host_3', 'request_cnt', 'part_of_day', 'event_time']

trans_save = trans
for col in tqdm.tqdm(cols):
    trans_save[col] = trans_save[col].apply(np.array)

trans_save.to_parquet('./data/trans_split.pq', engine='pyarrow')

100%|█████████████████████████████████████████████████████████████████████████████████| 6/6 [00:17<00:00,  2.83s/it]


CPU times: user 57.4 s, sys: 21.1 s, total: 1min 18s
Wall time: 1min 17s
